In [0]:
from pyspark.sql.functions import *
from datetime import datetime
from pyspark.sql.types import *

In [0]:
currentDate = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(currentDate)

In [0]:
# Defining the varibales
# tableName = "Healthverity"
inputPath = "/Volumes/source/incremental/Healthverity"
outputPath = "/Volumes/semantic_layer/incremental/Healthverity"
# metadataPath = "/Volumes/semantic_layer/incremental/incremental_catalog"
# currentDate = datetime.now().strftime("%Y-%m-%d")
# incrementalField = "date_service"
fullLoadFlag = False

In [0]:
pathCheck = True
try:
    dbutils.fs.ls("/Volumes/semantic_layer/incremental/Healthverity")
except:
    pathCheck = False
    pass

In [0]:
print(pathCheck)

In [0]:
# Read the output directory and get max date service value
if pathCheck == True:
    previousOutputDf = spark.read.format("delta").load(outputPath)
    maxDateService = previousOutputDf.select(functions.max('date_service')).collect()[0][0]
else:
    fullLoadFlag = True
    maxDateService = "2019-01-01"

In [0]:
print(maxDateService)

In [0]:
def data_load(loadFlag,inputPath,dateValue):
    if loadFlag == True:
        print("Full Load")
        sourceDf = spark.read.format("delta").load(inputPath)
    else:
        print("Incremental Load")
        sourceDf = spark.read.format("delta").load(inputPath) \
                    .filter(col("date_service") > dateValue)
    return sourceDf


In [0]:
finalDf = data_load(fullLoadFlag,inputPath,maxDateService)

In [0]:
display(finalDf.count())

In [0]:
finalDf.write.format("delta").mode("append").save(outputPath)

In [0]:
# metadataSchema = StructType([
#     StructField('tableName', StringType(), False),
#     StructField('incrementalField', StringType(), False),
#     StructField('incrementalValue', StringType(), False),
#     StructField('Load_Date', StringType(), False)
# ])

# metadata = [[tableName, incrementalField, maxDateService, currentDate]]
# metadataDf = spark.createDataFrame(metadata, metadataSchema)

In [0]:
%sql
select count(*) from delta.`/Volumes/semantic_layer/incremental/Healthverity`